# Robot stuff

In this notebook we can check how to sample the trajectory of the robot, how to get velocity profiles from the trajectory, which can be used to control the robot, etc.  

In [ ]:
import math
import matplotlib.pyplot as plt
import numpy as np
%matplotlib notebook
%reload_ext autoreload
%autoreload 2

np.set_printoptions(precision=2)

### Trajectory sampled uniformly in time

In [ ]:
from trajectory import Trajectory
from environment import Environment

from measurements import create_mask, get_D_topright
from solvers import alternativePseudoInverse, exactSolution

n_anchors = 4 #number of anchors
n_complexity = 3 #complexity of trajectory
dim = 2 # dimension of setup. 
n_positions = 10
model = 'bandlimited'
#model = 'polynomial'

trajectory = Trajectory(n_complexity, dim=dim, model=model)
trajectory.set_coeffs(seed=2)
environment = Environment(n_anchors)
environment.set_random_anchors(seed=2)

times_nonunif = trajectory.get_times(n_samples=n_positions)
basis_nonunif = trajectory.get_basis(times=times_nonunif)
sample_points_nonunif = trajectory.get_sampling_points(basis=basis_nonunif)

plt.figure()
environment.plot()
trajectory.plot(basis_nonunif, color='orange', label='ground truth')
plt.legend()

### Trajectory sampled uniformly in distance
After talking with Karen I decided to just hack a numerical ingegration. We can think of improving it later, 
for example having some adaptive steps, but for now it seems OK. We could also discuss sampling according to the curvature.

In [ ]:
def get_times_unform_in_path(n_samples, trajectory, time_steps=10000, plot=False):
    '''Calculate numerically times equivalent to uniform sampling in distance traveled.
    
    It calculates the cumulative integral over small steps, and picks as a sample time the frist time after 
    the integral reaches expected distance at this step.'''
    times = trajectory.get_times(n_samples=time_steps)
    basis_prime = trajectory.get_basis_prime(times=times)
    velocities = trajectory.coeffs.dot(basis_prime)

    time_differences = times[1:]-times[:-1]
    speeds = np.linalg.norm(velocities, axis=0)
    distances = np.cumsum((speeds[1:] + speeds[:-1])/2*time_differences)
    
    uniform_distances = np.arange(n_samples)*distances[-1]/(n_samples - 1)
    uniform_path_times = []
    errors = []
    i = 0
    for n in range(n_samples):
        while i < len(distances) and distances[i] < uniform_distances[n]:
            i = i + 1
        errors.append(distances[i] - uniform_distances[n])
        uniform_path_times.append(times[i])
        i = i + 1

    if plot:
        plt.figure()
        plt.plot(times[1:], distances, label="uniform in time")
        plt.plot(uniform_path_times, uniform_distances, "-*", label="uniorm in distance")
        plt.title("distance traveled")
        plt.legend()
        plt.show()
        
    return np.array(uniform_path_times), uniform_distances, np.array(errors)

In [ ]:
from trajectory import Trajectory
from environment import Environment

from measurements import create_mask, get_D_topright
from solvers import alternativePseudoInverse, exactSolution

times_unif, distances, errors = get_times_unform_in_path(n_samples=n_positions, 
                                                    trajectory=trajectory, 
                                                    time_steps=50000,
                                                    plot=True)
print("cumulative error: {}".format(np.linalg.norm(errors)))
basis_unif = trajectory.get_basis(times=times_unif)
sample_points_unif = trajectory.get_sampling_points(basis=basis_unif)

plt.figure()
environment.plot()
trajectory.plot(basis_nonunif, color='blue', label='non uniform')
trajectory.plot(basis_unif, color='orange', marker='x', label='uniform')
plt.legend()

## Compute the curvature at each point. 

For now there is still a problem. Since the points are not resampled in time, there is not only a radial acceleration (the one that makes the robot turn) but also a tangential radiation. In plot below, the orange lines showing the curvature radius, are therefore oriented in the direction of the curve. 

For now I am not quite sure if this is a problem, or if we can still try to incorporate this in the controls for the robot. 

In [ ]:
from matplotlib.patches import Circle

def get_curvatures(trajectory, times, ax=None):
    basis = trajectory.get_basis(times=times)
    sample_points = trajectory.get_sampling_points(basis=basis)
    
    # compute derivatives of the basis vectors.
    basis_prime = trajectory.get_basis_prime(times=times)
    basis_twoprime = trajectory.get_basis_twoprime(times=times)

    # see wikipedia for these definitions: 
    # https://en.wikipedia.org/wiki/Curvature 
    # Section: Precise definition
    tangents = trajectory.coeffs.dot(basis_prime)
    unit_tangents = tangents / np.linalg.norm(tangents, axis=0)
    curvatures = trajectory.coeffs.dot(basis_twoprime) 
    normal_vectors = curvatures / np.linalg.norm(curvatures, axis=0)

    # extract only radial component of acceleration 
    radial_acc = []
    for norm, unit in zip(normal_vectors.T, unit_tangents.T):
        norm = np.r_[norm.reshape((-1, )), 0] # create 3D
        unit = np.r_[unit.reshape((-1, )), 0] # create 3D
        radial = np.cross(unit, np.cross(norm, unit))
        radial_acc.append(radial[:2])
    radial_acc = np.array(radial_acc).T
    radii = 1 / np.linalg.norm(radial_acc, axis=0)
    radial_norm = radial_acc * radii
    
    # plot
    if ax is not None: 
        for i in range(1, sample_points.shape[1]-1):
            s = sample_points[:, i]
            t = s + unit_tangents[:, i]
            n = s + normal_vectors[:, i]
            r = s + radial_acc[:, i]
            plt.plot([s[0], t[0]], [s[1], t[1]], ':', color='black')
            plt.plot([s[0], n[0]], [s[1], n[1]], ':', color='orange')
            plt.plot([s[0], r[0]], [s[1], r[1]], ':', color='blue')

            radius = radii[i]
            center = s + radius * radial_norm[:, i]
            circ = Circle(xy=center, radius=radius, alpha=0.1, color='blue')
            plt.scatter(*center, color='blue', marker='+')
            plt.gca().add_artist(circ)
    
    return radii, unit_tangents, radial_acc

fig, ax = plt.subplots()
trajectory.plot(basis_nonunif, color='orange', label='ground truth')
xlims = ax.get_xlim()
ylims = ax.get_ylim()
plt.axis('equal')

radii, unit_tangents, radial_acc = get_curvatures(trajectory, times_nonunif, ax=ax)
ax.set_xlim(xlims)
ax.set_ylim(ylims)
#plt.xlim([-2, 8])
#plt.ylim([-2, 5])

In [ ]:
# TODO(FD) Below needs some refactoring, 
# there is a lot of duplicate code. 
def get_left_and_right_points(trajectory, times, width=0.5, ax=None):
    basis = trajectory.get_basis(times=times)
    sample_points = trajectory.get_sampling_points(basis=basis)
    
    # compute derivatives of the basis vectors.
    basis_prime = trajectory.get_basis_prime(times=times)
    basis_twoprime = trajectory.get_basis_twoprime(times=times)

    # see wikipedia for these definitions: 
    # https://en.wikipedia.org/wiki/Curvature 
    # Section: Precise definition
    tangents = trajectory.coeffs.dot(basis_prime)
    unit_tangents = tangents / np.linalg.norm(tangents, axis=0)
    curvatures = trajectory.coeffs.dot(basis_twoprime) 
    normal_vectors = curvatures / np.linalg.norm(curvatures, axis=0)

    # extract only radial component of acceleration 
    radial_acc = []
    for norm, unit in zip(normal_vectors.T, unit_tangents.T):
        norm = np.r_[norm.reshape((-1, )), 0]
        unit = np.r_[unit.reshape((-1, )), 0]
        radial = np.cross(unit, np.cross(norm, unit))
        radial_acc.append(radial[:2])
    radial_acc = np.array(radial_acc).T
    radii = 1 / np.linalg.norm(radial_acc, axis=0)
    radial_norm = radial_acc * radii
    
    points_left = sample_points - width / 2. * radial_norm
    points_right = sample_points + width / 2. * radial_norm
    
    label = 'left and right wheel'
    if ax is not None: 
        for cl, cr in zip(points_left.T, points_right.T):
            if all(np.isnan(cl)):
                continue
            plt.scatter(*cl, color='red', marker='+', label=label)
            label = None
            plt.scatter(*cr, color='red', marker='+', label=label)
    
    return points_left, points_right 

fig, ax = plt.subplots()
trajectory.plot(basis_unif)
plt.axis('equal')

ROBOT_WIDTH = 1.0
points_left, points_right = get_left_and_right_points(trajectory, times_unif, width=ROBOT_WIDTH, ax=ax)

In [ ]:
import csv

FOLDER = 'controls/'

def generate_control_parameters(seed, n_complexity, model, n_samples, fname=''):
    width = 0.3 # width of the robot
    
    trajectory = Trajectory(n_complexity=n_complexity, model=model) 
    trajectory.set_coeffs(seed=seed, dimension=2)
    times, distances, errors = get_times_unform_in_path(n_samples=n_samples, 
                                                        trajectory=trajectory, plot=False)
    
    basis = trajectory.get_basis(times=times)
    sample_points = trajectory.get_sampling_points(basis=basis)
    
    fig, ax = plt.subplots()
    trajectory.plot(basis, color='orange', label='trajectory')
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    xlims = ax.get_xlim()
    ylims = ax.get_ylim()
    plt.axis('equal')
    
    radii, __, __ = get_curvatures(trajectory=trajectory, times=times, ax=None)
    pos_left, pos_right = get_left_and_right_points(trajectory=trajectory, times=times, width=width, ax=ax)
    ax.set_xlim(xlims)
    ax.set_ylim(ylims)
    ax.legend()
    plt.savefig(FOLDER + fname + '.png')
    print('wrote plots as', FOLDER + fname + '.png')
    
    if fname[-4:] != '.txt':
        fnametxt = fname + '.txt'
    else:
        fnametxt = fname
        fname = fnametxt[:-4]
    with open(FOLDER + fnametxt, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile, delimiter=',')
        header = ['i', 'rad', 'dist', 'x', 'y', 'x_left', 'y_left', 'x_right', 'y_right']
        #print(header)
        writer.writerow(header)
        for i in range(len(distances) - 1):
            r = radii[i + 1]
            delta_d = distances[i + 1]  - distances[i]
            p = sample_points[:, i]
            pl = pos_left[:, i]
            pr = pos_right[:, i]
            line = [i, r, delta_d, p[0], p[1], pl[0], pl[1], pr[0], pr[1]]
            #print(np.array(line))
            writer.writerow(line)
        print('wrote parameters as', FOLDER + fnametxt)
    

trajectories = [
    dict(
        fname = 'bandlimited1',
        model = 'bandlimited',
        n_complexity = 3,
        n_samples = 50,
        seed = 2
    ), 
    dict(
        fname = 'polynomial1',
        model = 'polynomial',
        n_complexity = 3,
        n_samples = 10,
        seed = 1
    ), 
]

for traj in trajectories:
    generate_control_parameters(**traj)